In [2]:
import torch
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [3]:
class ShapesDatasets(Dataset):
    def __init__(self, roor_dir, transform=None):
        self.datasets = datasets.ImageFolder(roor=roor_dir, transform=transform)
    
    def __len__(self):
        return len(self.datasets)
    
    def __getitem__(self, idx):
        return self.datasets[idx]